<a href="https://colab.research.google.com/github/AlexanderF048/GOIT_DS/blob/main/hw_10_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import tensorflow

from tensorflow import keras
from keras.datasets import fashion_mnist

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
from keras import losses
from keras import metrics

import keras.backend

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils import img_to_array, array_to_img, to_categorical

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
#x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.figure(figsize=(7.5,7.5))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3),
           activation="relu",
           input_shape=(28, 28, 1),
           dilation_rate=(1, 1),
           padding="same",
           ),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3),
           activation="relu",
           dilation_rate=(1, 1),
           padding="same",
           ),
    MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3),
           activation="relu",
           dilation_rate=(1, 1),
           padding="same",
           ),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation="relu"),

    Dense(10, activation="softmax")
])

model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(learning_rate=0.001),
    loss= losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=8,
    batch_size=30,
    validation_split=0.2, #20% на тестовые экземпляры
)

print('---------------------------------------------------------------------------------------------------------------------')
print(history.history.keys())

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()
val_acc_values = history_dict['val_accuracy']
plt.plot(epochs, history_dict['accuracy'], 'bo', label='Training acc')
plt.plot(epochs, history_dict['val_accuracy'], 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
result = model.predict(x_test[:1])
print(result)
plt.imshow(x_test[:1].reshape(28, 28, 1))

**VGG16**

---



In [ ]:
conv_base = VGG16(weights="imagenet", include_top=False,
                  input_shape=(48, 48, 3),
                  classes=784,
    classifier_activation="relu")
conv_base.trainable = False
conv_base.summary()

In [ ]:
# Reshape images as per the tensor format required by tensorflow
X_train = x_train
print(X_train[1].shape)
print(X_train.shape)
print("------------------------------------------------------------------------")

X_train = X_train.reshape([-1, 784]) # Возвращаем в array
print(X_train.shape)
print("------------------------------------------------------------------------")

X_train = np.dstack([X_train] * 3) # 2D в 3D
print(X_train.shape)
print("------------------------------------------------------------------------")

X_train = X_train.reshape(-1, 28,28,3)
print(X_train.shape)

print("------------------------------------------------------------------------")
X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
print(X_train[1].shape)
print(X_train.shape)


In [ ]:
X_train = preprocess_input(X_train)

In [ ]:
#Y_train = to_categorical(y_train)

In [ ]:
modelVGG16 = Sequential([
    conv_base,

    Flatten(),
    Dense(128, activation="relu"),

    Dense(10, activation="softmax")
])

modelVGG16.summary()

In [ ]:
modelVGG16.compile(
    optimizer=optimizers.RMSprop(learning_rate=0.001),
    loss= losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block1_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
historyVGG16 = modelVGG16.fit(
    X_train,
    y_train,
    epochs=50,
    steps_per_epoch=100,
    validation_steps=50,
    batch_size=30,
    validation_split=0.2, #20% на тестовые экземпляры
)

print('---------------------------------------------------------------------------------------------------------------------')
print(historyVGG16.history.keys())